### Joel Greenblat's Magic Formula 

In [1]:
# Importação das bibliotecas
import numpy as np
import pandas as pd
import requests

In [2]:
# Extrai o banco de dados da página 'Fundamentus'
url     = ('https://www.fundamentus.com.br/resultado.php')
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
formData = {
    'pl_min': '',
    'pl_max': '',
    'pvp_min': '',
    'pvp_max': '',
    'psr_min': '',
    'psr_max': '',
    'divy_min': '',
    'divy_max': '',
    'pativos_min': '',
    'pativos_max': '',
    'pcapgiro_min': '',
    'pcapgiro_max': '',
    'pebit_min': '',
    'pebit_max': '',
    'fgrah_min': '',
    'fgrah_max': '',
    'firma_ebit_min': '',
    'firma_ebit_max': '',
    'firma_ebitda_min': '',
    'firma_ebitda_max': '',
    'margemebit_min': '',
    'margemebit_max': '',
    'margemliq_min': '',
    'margemliq_max': '',
    'liqcorr_min': '',
    'liqcorr_max': '',
    'roic_min': '',
    'roic_max': '',
    'roe_min': '',
    'roe_max': '',
    'liq_min': '',
    'liq_max': '',
    'patrim_min': '',
    'patrim_max': '',
    'divbruta_min': '',
    'divbruta_max': '',
    'tx_cresc_rec_min': '',
    'tx_cresc_rec_max': '',
    'setor': '',
    'negociada': 'ON',
    'ordem': '1',
    'x': '35',
    'y': '20',
}
# res     = requests.get(url, headers=headers, data=formData)
res     = requests.post(url, headers=headers, data=formData)
df      = pd.read_html(res.content)[0]
df.head(5)

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,GFSA3,297,-63890.0,56,794,"0,00%",227,77.00000,987.0,260,...,1603.0,"8,04%","-0,10%",175,"2,65%","-0,09%",19.843.20000,1.630.560.00000,78,"18,80%"
1,DASA3,4594,-41677.0,346,2713,"0,55%",1416,875.00000,61013.0,-865,...,3077.0,"0,44%","-0,56%",161,"0,33%","-0,83%",10.292.00000,7.265.200.00000,79,"27,54%"
2,MAPT4,4679,-39091.0,-732,0,"0,00%",55627,-3.90909,-45745.0,-647,...,-45745.0,"0,00%","0,00%",0,"-12,34%","1,87%",2.01691,-5.876.00000,0,"0,00%"
3,WEST3,581,-26388.0,159,2344,"0,00%",1204,182.00000,-2685.0,199,...,-1510.0,"-8,73%","-0,89%",462,"-23,24%","-0,60%",5.383.59000,403.271.00000,8,"0,00%"
4,SBFG3,3227,-25548.0,413,2179,"0,00%",1183,617.00000,6497.0,-556,...,2433.0,"3,35%","-0,85%",162,"2,20%","-1,62%",28.203.10000,1.898.800.00000,118,"30,19%"


In [3]:
# Converte coluna ROIC de valores string para float
df['ROIC'] = df['ROIC'].str.replace(',','.').str.rstrip('%').astype('float');

df['Codigo'] = df['Papel'].str.extract('(\d)', expand=True).astype('int') # Separa o Tipo/Código em outra coluna (ordinária, preferencial, etc..)
df['Papel'] = df['Papel'].str.rstrip('1234567890') # Pega somente Letras do papel
df.head(5)

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Codigo
0,GFSA,297,-63890.0,56,794,"0,00%",227,77.00000,987.0,260,...,"8,04%","-0,10%",175,2.65,"-0,09%",19.843.20000,1.630.560.00000,78,"18,80%",3
1,DASA,4594,-41677.0,346,2713,"0,55%",1416,875.00000,61013.0,-865,...,"0,44%","-0,56%",161,0.33,"-0,83%",10.292.00000,7.265.200.00000,79,"27,54%",3
2,MAPT,4679,-39091.0,-732,0,"0,00%",55627,-3.90909,-45745.0,-647,...,"0,00%","0,00%",0,-12.34,"1,87%",2.01691,-5.876.00000,0,"0,00%",4
3,WEST,581,-26388.0,159,2344,"0,00%",1204,182.00000,-2685.0,199,...,"-8,73%","-0,89%",462,-23.24,"-0,60%",5.383.59000,403.271.00000,8,"0,00%",3
4,SBFG,3227,-25548.0,413,2179,"0,00%",1183,617.00000,6497.0,-556,...,"3,35%","-0,85%",162,2.20,"-1,62%",28.203.10000,1.898.800.00000,118,"30,19%",3


In [4]:
# Tabela contendo os setores de cada papel
df_setorial = pd.read_csv('setorial.csv')
df_setorial = df_setorial[df_setorial["SEGMENTO.1"].notna()]
df_setorial.drop(['SEGMENTO.1'], axis=1, inplace = True)


df_final = df.join(df_setorial.set_index('CÓDIGO'), on='Papel')
df_final['SETOR ECONÔMICO'].replace(np.nan, 'Outros', inplace=True)
df_final['SETOR ECONÔMICO'].unique()
# df_final

array(['Consumo Cíclico', 'Saúde', 'Outros', 'Tecnologia da Informação',
       'Utilidade Pública', 'Bens Industriais', 'Comunicações',
       'Financeiro', 'Petróleo, Gás e Biocombustíveis',
       'Materiais Básicos', 'Consumo não Cíclico'], dtype=object)

In [5]:
df_final.set_index('SETOR ECONÔMICO', inplace = True)
df_final = df_final.drop('Financeiro', axis=0)
# df_final.reset_index()['SETOR ECONÔMICO'].unique()
df_final.reset_index(inplace = True)

In [6]:
# Seleciona valores maiores que 0
selecao = (df_final['EV/EBIT'] > 0) & (df_final['ROIC'] > 0) 
df2 = df_final[selecao].copy(deep=True)

In [7]:
# RANK EV/EBIT
df2.sort_values(by="EV/EBIT", ascending=True, inplace=True) # Sort Crescente
df2['Rank_EV'] = range(1,df2.shape[0]+1) # insere os indices do Rank

df2[['Papel','EV/EBIT','Rank_EV']].head(5)

,Papel,EV/EBIT,Rank_EV
80,HETA,1.01396,1
73,HETA,1.14233,2
147,GOAU,176.00000,3
356,MNPR,180.00000,4
167,GOAU,189.00000,5


In [8]:
# RANK ROIC
df2.sort_values(by="ROIC", ascending=False, inplace=True) # Sort Decrescente
df2['Rank_ROIC'] = range(1,df2.shape[0]+1) # Insere os Indices do Rank

df2[['Papel','ROIC','Rank_ROIC']].head(5)

,Papel,ROIC,Rank_ROIC
250,ATOM,82.04,1
146,CMIN,81.03,2
272,WHRL,55.87,3
310,WHRL,55.87,4
136,TASA,49.06,5


In [9]:
# RANK Magic
df2['Rank_Soma'] = df2['Rank_ROIC'] + df2['Rank_EV'] # Somatório dos Ranks

df2 = df2.sort_values(by="Rank_Soma") # Sort dos Ranks

df2['Rank_Magic'] = range(1,df2.shape[0]+1) # Insere indices

In [10]:
# Mostra a tabela
selecao = ['Papel','Codigo','EV/EBIT','Rank_EV','ROIC','Rank_ROIC','Rank_Soma','Rank_Magic'] # Seleciona apenas colunas

df2[selecao].head(20) # 20 primeiros papeis

,Papel,Codigo,EV/EBIT,Rank_EV,ROIC,Rank_ROIC,Rank_Soma,Rank_Magic
146,CMIN,3,224.0,6,81.03,2,8,1
169,VALE,3,272.0,12,44.97,7,19,2
168,PLPL,3,262.0,11,41.15,11,22,3
122,CSNA,3,293.0,13,40.20,12,25,4
124,BRKM,6,243.0,7,33.38,24,31,5
135,ETER,3,373.0,24,39.90,13,37,6
120,USIM,3,246.0,9,27.33,31,40,7
121,USIM,5,250.0,10,27.33,30,40,8
133,TASA,4,463.0,34,49.06,6,40,9
130,USIM,6,321.0,14,27.33,29,43,10


In [11]:
# Tabela contendo os setores de cada papel
df_setorial = pd.read_csv('setorial.csv')

df_setorial.head(5)

,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,EMPRESA,CÓDIGO,SEGMENTO.1
0,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",3R PETROLEUM,RRRP,NM
1,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",COSAN,CSAN,NM
2,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",DOMMO,DMMO,NaN
3,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",ENAUTA PART,ENAT,NM
4,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",PET MANGUINH,RPMG,NaN


In [12]:
# Remove linhas vazias (problema gerado na hora de crirar o csv e corrigido aqui)
df_setorial2 = df_setorial[df_setorial["SEGMENTO.1"].notna()]

df_setorial2.head(5)

,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,EMPRESA,CÓDIGO,SEGMENTO.1
0,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",3R PETROLEUM,RRRP,NM
1,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",COSAN,CSAN,NM
3,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",ENAUTA PART,ENAT,NM
5,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",PETROBRAS,PETR,N2
6,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",PETROBRAS BR,BRDT,NM


In [13]:
# df6 = df5.copy() # Copia do df5
# df6['Papel'] = df5['Papel'].str.rstrip('1234567890') # Pega somente Letras do papel
# df6['Codigo'] = df5['Papel'].str.extract('(\d)', expand=True).astype('int') # Separa o Tipo/Código em outra coluna (ordinária, preferencial, etc..)
# df6.head(5)

In [14]:
# Próximo passo: PUXAR O SETOR DO df_setorial2 com base no 'Papel' do df6